In [2]:
import pandas as pd
import torch
from transformers import BertTokenizer, AutoTokenizer, BertModel, BertConfig, AutoModel, AdamW
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader



/Users/schen/.pyenv/versions/3.7.8/envs/wecloud378/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


In [ ]:
df = pd.read_csv('../data/dataset.csv')

In [ ]:
vocabs = []
for d in df['label']:
  x_list = ast.literal_eval(d)
  vocabs.append(set(x_list))
print(vocabs)

In [ ]:
mlb = MultiLabelBinarizer()
preds = mlb.fit_transform(vocabs)
print(preds)

In [242]:
y = df['label'].apply(ast.literal_eval)
y = y.apply(lambda x: [set(x)])
y = y.apply(mlb.transform)
y = y.apply(lambda x: x[0])
print(y)

In [244]:
x = df['title']
y = y
df_cpy = df

In [259]:
dummies = df_cpy['label'].apply(lambda x: eval(x)).str.join('|').str.get_dummies()
LABEL_COLUMNS = dummies.columns
print(LABEL_COLUMNS)

In [260]:
print(dummies)

,ai,algorithm,algorithms,application,applications,bandits,bayesian,best practice,best practices,big data,...,theory,tools,trading,tutorial,tutorials,use case,use cases,ux,visualization,web development
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
329,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
330,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
331,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [264]:
df_final = pd.merge(df_cpy, dummies, left_index=True, right_index=True).dropna()
print(df_final)

,title,label,ai,algorithm,algorithms,application,applications,bandits,bayesian,best practice,...,theory,tools,trading,tutorial,tutorials,use case,use cases,ux,visualization,web development
0,Four Steps To Turn Big Data Into Action,['big data'],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9 Amazing Ways Big Data Is Used Today to Chang...,['big data'],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Data Science and Moneyball: A Profile of Pete ...,['data science'],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Stanford Algorithm Analyzes Sentence Sentiment...,['machine learning'],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Machine Learning for Relevance and Serendipity,['machine learning'],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328,Word2Vec Skip The Gram Tutorial,"['nlp', 'machine learning']",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
329,150 successful machine learning models: 6 less...,"['machine learning', 'application']",0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
330,Introduction to Adversarial Machine Learning,['machine learning'],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
331,The Craft of Writing Effectively,['career'],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [448]:
# First Split for Train and Test
train, test = train_test_split(df_final, random_state=42, test_size=0.30, shuffle=True)
train, val = train_test_split(train, test_size=0.2, random_state=42,shuffle=True)
print(train.shape, test.shape, val.shape)

(184, 91) (99, 91) (46, 91)


In [449]:
train[LABEL_COLUMNS].sum()

ai                 5
algorithm          9
algorithms         1
application        5
applications       0
                  ..
use case           3
use cases          1
ux                 2
visualization      2
web development    6
Length: 89, dtype: int64

In [450]:
train[LABEL_COLUMNS].sum().sum()

246

In [452]:
sample_row = train.iloc[16]
sample_comment = sample_row.title
sample_labels = sample_row[LABEL_COLUMNS]

print(sample_comment)
print()
print(sample_labels.to_dict())

This Amazing Image Algorithm Learns to Spot Objects Without Human Help

{'ai': 0, 'algorithm': 0, 'algorithms': 0, 'application': 0, 'applications': 0, 'bandits': 0, 'bayesian': 0, 'best practice': 0, 'best practices': 0, 'big data': 0, 'book': 0, 'books': 0, 'business': 0, 'career': 0, 'cloud': 0, 'clustering': 0, 'computer science': 0, 'computer vision': 1, 'course': 0, 'courses': 0, 'dashboard': 0, 'data': 0, 'data engineering': 0, 'data mining': 0, 'data science': 0, 'data shift': 0, 'data visualization': 0, 'dataset': 0, 'decision making': 0, 'deep learning': 0, 'design patterns': 0, 'designed patterns': 0, 'engineering': 0, 'experimentation': 0, 'forecasting': 0, 'gis': 0, 'graph': 0, 'guide': 0, 'hardware': 0, 'healthcare': 0, 'hiring': 0, 'how-to': 0, 'improvement': 0, 'industry': 0, 'infrastructure': 0, 'interview': 0, 'interviews': 0, 'iot': 0, 'libraries': 0, 'library': 0, 'machine learning': 0, 'mathematics': 0, 'mlops': 0, 'mobile development': 0, 'neural network': 0, 'nlp

In [453]:
# loading tokenizer of bert base version
BERT_MODEL_NAME = "bert-base-cased"
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

encoding = tokenizer.encode_plus(
    sample_comment, 
    add_special_tokens=True,
    max_length=512,
    return_token_type_ids=False,
    padding='max_length',
    return_attention_mask=True,
    return_tensors="pt"
)

encoding.keys()

dict_keys(['input_ids', 'attention_mask'])

In [454]:
print(encoding["input_ids"].shape, encoding["attention_mask"].shape)
print(encoding["input_ids"].squeeze()[:20])
print(encoding["attention_mask"].squeeze()[:20])
print(encoding["input_ids"].squeeze().shape)

(torch.Size([1, 512]), torch.Size([1, 512]))

In [458]:
tokenizer.convert_ids_to_tokens(encoding["input_ids"].squeeze())[:20]

['[CLS]',
 'This',
 'Amazing',
 'Image',
 'Al',
 '##gor',
 '##ith',
 '##m',
 'Lea',
 '##rns',
 'to',
 'Spot',
 'O',
 '##bject',
 '##s',
 'Without',
 'Human',
 'Help',
 '[SEP]',
 '[PAD]']

In [459]:
class TitleDataset(Dataset):

    def __init__(self, data: pd.DataFrame, tokenizer: BertTokenizer, max_token_len: int = 128):

        self.data = data
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        
        data_row = self.data.iloc[index]

        title = data_row.title
        labels = data_row[LABEL_COLUMNS]

        encoding = self.tokenizer.encode_plus(
            title,
            add_special_tokens=True,
            max_length=self.max_token_len,
            return_token_type_ids=False,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_tensors="pt"
        )
        
        return dict(
            title=title,
            input_ids=encoding["input_ids"].flatten(),
            attention_mask=encoding["attention_mask"].flatten(),
            labels=torch.FloatTensor(labels)
        )

In [460]:
train_dataset = TitleDataset(train, tokenizer)


In [461]:
sample_item = train_dataset[0]


In [462]:
sample_item.keys()


dict_keys(['title', 'input_ids', 'attention_mask', 'labels'])

In [463]:
sample_item["title"]


'Algorithmic Trading Models - Machine Learning'

In [464]:
sample_item["labels"]


tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [465]:
sample_item["input_ids"].shape


torch.Size([128])

In [466]:
bert_model = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [467]:
sample_item["input_ids"].unsqueeze(dim=0).shape


torch.Size([1, 128])

In [468]:
prediction = bert_model(sample_item["input_ids"].unsqueeze(dim=0), sample_item["attention_mask"].unsqueeze(dim=0))


In [469]:
prediction.last_hidden_state.shape, prediction.pooler_output.shape


(torch.Size([1, 128, 768]), torch.Size([1, 768]))

In [470]:
import pytorch_lightning as pl

class TitleDataModule(pl.LightningDataModule):
    def __init__(self, train_df, test_df, tokenizer, batch_size=0, max_token_len=128):
        super().__init__
        self.train_df = train_df
        self.test_df = test_df
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        self.max_token_len = max_token_len
        self.prepare_data_per_node = True
        self._log_hyperparams = True

    def setup(self, stage):
        self.train_dataset = TitleDataset(
            self.train_df,
            self.tokenizer,
            self.max_token_len
        )

        self.test_dataset = TitleDataset(
            self.test_df,
            self.tokenizer,
            self.max_token_len
        )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4
        )

    def val_dataloader(self):
        return DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=4
        )

    def val_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=1, num_workers=4)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=1, num_workers=4)

In [471]:
# MAX_LEN = 200
# TRAIN_BATCH_SIZE = 64
# VALID_BATCH_SIZE = 64
# EPOCHS = 10
# LEARNING_RATE = 2e-5
# tokenizer = AutoTokenizer.from_pretrained('roberta-base')

In [472]:
N_EPOCHS = 10
BATCH_SIZE = 32

data_module = TitleDataModule(train, test, tokenizer, batch_size=BATCH_SIZE)
data_module.setup(stage="None")

In [473]:
import torch.nn as nn

criterion = nn.BCELoss()

prediction = torch.FloatTensor(
    [10.95873564, 1.07321467, 1.58524066, 0.03839076, 15.72987556, 1.09513213]
)

labels = torch.FloatTensor(
  [1., 0., 0., 0., 1., 0.]
)

In [474]:
torch.sigmoid(prediction)

tensor([1.0000, 0.7452, 0.8299, 0.5096, 1.0000, 0.7493])

In [475]:
output = criterion(torch.sigmoid(prediction), labels)
output

tensor(0.8725)

In [476]:
from torchmetrics.functional import accuracy, f1_score, auroc
from transformers import BertTokenizerFast as BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup

class TitleTagger(pl.LightningModule):

  def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
    super().__init__()
    self.bert = BertModel.from_pretrained(BERT_MODEL_NAME, return_dict=True)
    self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    self.criterion = nn.BCELoss()

  def forward(self, input_ids, attention_mask, labels=None):
    output = self.bert(input_ids, attention_mask=attention_mask)
    output = self.classifier(output.pooler_output)
    output = torch.sigmoid(output)    
    loss = 0
    if labels is not None:
        loss = self.criterion(output, labels)
    return loss, output

  def training_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss, "predictions": outputs, "labels": labels}

  def validation_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    labels = batch["labels"]
    loss, outputs = self(input_ids, attention_mask, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def training_epoch_end(self, outputs):
    
    labels = []
    predictions = []
    for output in outputs:
      for out_labels in output["labels"].detach().cpu():
        labels.append(out_labels)
      for out_predictions in output["predictions"].detach().cpu():
        predictions.append(out_predictions)

    labels = torch.stack(labels).int()
    predictions = torch.stack(predictions)

    for i, name in enumerate(LABEL_COLUMNS):
      class_roc_auc = auroc(predictions[:, i], labels[:, i])
      self.logger.experiment.add_scalar(f"{name}_roc_auc/Train", class_roc_auc, self.current_epoch)


  def configure_optimizers(self):

    optimizer = AdamW(self.parameters(), lr=2e-5)

    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=self.n_warmup_steps,
      num_training_steps=self.n_training_steps
    )

    return dict(
      optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )

In [477]:
steps_per_epoch=len(train) // BATCH_SIZE
total_training_steps = steps_per_epoch * N_EPOCHS

In [478]:
warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

(10, 50)

In [479]:
model = TitleTagger(
  n_classes=len(LABEL_COLUMNS),
  n_warmup_steps=warmup_steps,
  n_training_steps=total_training_steps 
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [480]:
import numpy as np
from tqdm.auto import tqdm
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger


In [481]:
checkpoint_callback = ModelCheckpoint(
  dirpath="checkpoints",
  filename="best-checkpoint",
  save_top_k=1,
  verbose=True,
  monitor="val_loss",
  mode="min"
)

In [482]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)


In [483]:
trainer = pl.Trainer(
  logger=True,
  callbacks=[early_stopping_callback, checkpoint_callback],
  max_epochs=N_EPOCHS)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [484]:
trainer.fit(model, data_module)


/Users/schen/.pyenv/versions/3.7.8/envs/wecloud378/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name       | Type      | Params
-----------------------------------------
0 | bert       | BertModel | 108 M 
1 | classifier | Linear    | 68.4 K
2 | criterion  | BCELoss   | 0     
-----------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.515   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | 

/Users/schen/.pyenv/versions/3.7.8/envs/wecloud378/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py:1898: PossibleUserWarning: The number of training batches (6) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  category=PossibleUserWarning,


Epoch 0:   0%|          | 0/105 [00:00<?, ?it/s] huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALL

/Users/schen/.pyenv/versions/3.7.8/envs/wecloud378/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)
Epoch 0, global step 6: 'val_loss' reached 0.68711 (best 0.68711), saving model to '/Users/schen/wecloud/Hacker-News-Scraper/checkpoints/best-checkpoint.ckpt' as top 1


...

Epoch 8:   0%|          | 0/105 [00:00<?, ?it/s, loss=0.513, v_num=3, train_loss=0.492, val_loss=0.485]          	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable t

Epoch 8, global step 54: 'val_loss' reached 0.48441 (best 0.48441), saving model to '/Users/schen/wecloud/Hacker-News-Scraper/checkpoints/best-checkpoint.ckpt' as top 1


Epoch 9:   0%|          | 0/105 [00:00<?, ?it/s, loss=0.499, v_num=3, train_loss=0.489, val_loss=0.484]          	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable t

Epoch 9, global step 60: 'val_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=10` reached.


Epoch 9: 100%|██████████| 105/105 [01:49<00:00,  1.04s/it, loss=0.492, v_num=3, train_loss=0.490, val_loss=0.484]


In [485]:
model.freeze()
model.eval()

TitleTagger(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)


In [488]:
# torch.save(model.state_dict(), 'model_2.pt')

In [489]:
test_comment = "Deep Learning course at UofT"



In [490]:
encoding = tokenizer.encode_plus(
  test_comment,
  add_special_tokens=True,
  max_length=512,
  return_token_type_ids=False,
  padding="max_length",
  return_attention_mask=True,
  return_tensors='pt',
)

In [491]:
_, test_prediction = model(encoding["input_ids"], encoding["attention_mask"])
test_prediction = test_prediction.flatten().numpy()

for label, prediction in zip(LABEL_COLUMNS, test_prediction):
  print(f"{label}: {prediction}")

ai: 0.5222846269607544
algorithm: 0.4106980264186859
algorithms: 0.45666205883026123
application: 0.2543446719646454
applications: 0.3394503891468048
bandits: 0.3104473948478699
bayesian: 0.3944703936576843
best practice: 0.38833609223365784
best practices: 0.4148646891117096
big data: 0.3509252667427063
book: 0.33761709928512573
books: 0.3325932025909424
business: 0.35062679648399353
career: 0.5535091757774353
cloud: 0.2445906698703766
clustering: 0.25908687710762024
computer science: 0.24501271545886993
computer vision: 0.2924898862838745
course: 0.3430578410625458
courses: 0.4456714391708374
dashboard: 0.2817031145095825
data: 0.19195318222045898
data engineering: 0.2928210198879242
data mining: 0.25372743606567383
data science: 0.393507182598114
data shift: 0.49364951252937317
data visualization: 0.39857810735702515
dataset: 0.2721661627292633
decision making: 0.3503306210041046
deep learning: 0.3849361836910248
design patterns: 0.301515132188797
designed patterns: 0.42623019218444

In [506]:
model

TitleTagger(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)


In [494]:
THRESHOLD = 0.5

test_comment = "Big Data at Air Miles"
encoding = tokenizer.encode_plus(
  test_comment,
  add_special_tokens=True,
  max_length=512,
  return_token_type_ids=False,
  padding="max_length",
  return_attention_mask=True,
  return_tensors='pt',
)

_, test_prediction = model(encoding["input_ids"], encoding["attention_mask"])
test_prediction = test_prediction.flatten().numpy()

for label, prediction in zip(LABEL_COLUMNS, test_prediction):
  if prediction < THRESHOLD:
    continue
  print(f"{label}: {prediction}")

career: 0.5700024962425232
data shift: 0.5138964653015137
machine learning: 0.5504406690597534
papers: 0.5153244733810425
robotics: 0.5299980640411377
simulation: 0.5706175565719604
system design: 0.5519585013389587
use case: 0.5647748708724976
ux: 0.6132348775863647
web development: 0.6017593145370483


In [495]:
MAX_TOKEN_COUNT = 512

In [496]:
vice = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
trained_model = model.to(device)

val_dataset = TitleDataset(
  val,
  tokenizer,
  max_token_len=MAX_TOKEN_COUNT
)

predictions = []
labels = []

for item in tqdm(val_dataset):
  _, prediction = trained_model(
    item["input_ids"].unsqueeze(dim=0).to(device), 
    item["attention_mask"].unsqueeze(dim=0).to(device)
  )
  predictions.append(prediction.flatten())
  labels.append(item["labels"].int())

predictions = torch.stack(predictions).detach().cpu()
labels = torch.stack(labels).detach().cpu()

100%|██████████| 46/46 [00:33<00:00,  1.37it/s]


In [497]:
accuracy(predictions, labels, threshold=THRESHOLD)


tensor(0.8801)

In [498]:
print("AUROC per tag")
for i, name in enumerate(LABEL_COLUMNS):
  tag_auroc = auroc(predictions[:, i], labels[:, i], pos_label=1)
  print(f"{name}: {tag_auroc}")

AUROC per tag
ai: 0.8222222328186035
algorithm: 0.5555555820465088
algorithms: 0.6022727489471436
application: 0.7596899271011353
applications: 0.0
bandits: 0.0
bayesian: 0.0
best practice: 0.0
best practices: 0.0
big data: 0.4545454680919647
book: 0.0
books: 0.0
business: 0.0
career: 0.19999998807907104
cloud: 0.0
clustering: 0.0
computer science: 0.511904776096344
computer vision: 0.738636314868927
course: 0.5227272510528564
courses: 0.0
dashboard: 0.0
data: 0.2888888716697693
data engineering: 0.0
data mining: 0.6666666269302368
data science: 0.27906978130340576
data shift: 0.0
data visualization: 0.0
dataset: 0.0
decision making: 0.9545454382896423
deep learning: 0.4728682041168213
design patterns: 0.0
designed patterns: 0.0
engineering: 0.0
experimentation: 0.0
forecasting: 0.7111110687255859
gis: 0.1111111044883728
graph: 0.35555553436279297
guide: 0.5333333015441895
hardware: 0.0
healthcare: 0.0
hiring: 0.0
how-to: 0.0
improvement: 0.8888888359069824
industry: 0.6222221851348877

/Users/schen/.pyenv/versions/3.7.8/envs/wecloud378/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score
  warnings.warn(*args, **kwargs)


In [500]:
from sklearn.metrics import classification_report
y_pred = predictions.numpy()
y_true = labels.numpy()

upper, lower = 1, 0

y_pred = np.where(y_pred > THRESHOLD, upper, lower)

print(classification_report(
  y_true, 
  y_pred, 
  target_names=LABEL_COLUMNS, 
  zero_division=0
))

                        precision    recall  f1-score   support

                    ai       0.04      1.00      0.07         1
             algorithm       0.00      0.00      0.00         1
            algorithms       0.00      0.00      0.00         2
           application       0.00      0.00      0.00         3
          applications       0.00      0.00      0.00         0
               bandits       0.00      0.00      0.00         0
              bayesian       0.00      0.00      0.00         0
         best practice       0.00      0.00      0.00         0
        best practices       0.00      0.00      0.00         0
              big data       0.00      0.00      0.00         2
                  book       0.00      0.00      0.00         0
                 books       0.00      0.00      0.00         0
              business       0.00      0.00      0.00         0
                career       0.02      1.00      0.04         1
                 cloud       0.00      

In [505]:
model

TitleTagger(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
